In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read Womens Clothing E-Commerce Reviews.csv:
import pandas as pd
df = pd.read_csv('../input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv')
df.info()

In [ ]:
# Only take the review and recommendation column
df = df.drop(df.columns[[0, 1, 2, 3, 5, 7, 8, 9, 10]], axis=1)
df = df.dropna()
df

Review Text: reviews of women's clothes. Recommended Id: 1 means Recommended, 0 means Not Recommended

In [ ]:
# Split data into train data and test data
from sklearn.model_selection import train_test_split
review = df['Review Text'].values
recommendation = df['Recommended IND'].values
review_train, review_test, recommendation_train, recommendation_test = train_test_split(review, recommendation, test_size=0.2)

In [ ]:
# Tokenization of review texts
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=50000, oov_token='x')
tokenizer.fit_on_texts(review_train) 
tokenizer.fit_on_texts(review_test)
 
sequence_train = tokenizer.texts_to_sequences(review_train)
sequence_test = tokenizer.texts_to_sequences(review_test)

padded_train = pad_sequences(sequence_train) 
padded_test = pad_sequences(sequence_test)

In [ ]:
# Neural network model with a sequential model
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=50000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# Training process
# In addition, if we want to display accuracy in each model training process we can add a metrics parameter and fill it with the string 'accuracy'.
num_epochs = 10
history = model.fit(padded_train, recommendation_train, epochs=num_epochs, 
                    validation_data=(padded_test, recommendation_test), verbose=2)

In [ ]:
# Plotting training accuracy and validation accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], 'r', label='Accuracy Training')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy Validation')
plt.title('Accuracy Training and Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc=0)
plt.show()

In [ ]:
# Plotting training loss and validation loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], 'r', label='Loss Training')
plt.plot(history.history['val_loss'], 'b', label='Loss Validation')
plt.title('Loss Training and Validation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc=0)
plt.show()

You can tinker with the Sequential Model part to get great results like adding the dropout() function.

In [ ]:
# You can try your review.
my_review = ["I really wanted to like this dress. the material, design, length and fit all over was perfect, except for the armholes- too big and the ruffles came right upto the nipple line. not comfortable with that. sadly, had to return."]
result = tokenizer.texts_to_sequences(my_review)
result = pad_sequences(result)

#print recommended or not
import numpy as np
result = model.predict_classes(result)
if result == [0]:
  print('Not Recommended')
elif result == [1]:
  print('Recommended')